In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from spec_id import Specz_fit
from glob import glob
from astropy.io import fits, ascii
from astropy.table import Table
from scipy.interpolate import interp1d
import seaborn as sea
sea.set(style='white')
sea.set(style='ticks')
sea.set_style({"xtick.direction": "in","ytick.direction": "in"})
colmap = sea.cubehelix_palette(12, start=2, rot=.2, dark=0, light=1.1, as_cmap=True)

In [18]:
# qgDB = pd.read_pickle('../data/quiescent_gal_DB.pkl')
gsDB = pd.read_pickle('../data/good_spec_gal_DB.pkl')

In [20]:
gsDB.columns

Index([u'gids', u'ids', u'redshift', u'hmag', u'ra', u'dec', u'agn',
       u'in_data', u'spec', u'flt_files', u'flt_xpos', u'flt_ypos',
       u'low_res_specz'],
      dtype='object')

In [17]:
# pd.to_pickle(gsDB,'../data/good_spec_gal_DB.pkl')

In [3]:
pz_list = glob('../rshift_dat/*Pofz.npy')

In [14]:
%matplotlib inline

lr_specz = np.zeros(len(pz_list))

for i in range(len(lr_specz)):
    z,pz=np.load(pz_list[i])
    lr_specz[i] = z[np.argmax(pz)]

In [15]:
gsDB['low_res_specz'] = lr_specz